## Import

In [73]:
import numpy as np
import pandas as pd

In [74]:
# pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)
# pd.set_option('display.expand_frame_repr', True)

In [75]:
import scipy
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import random
import datetime
import warnings
warnings.filterwarnings('ignore')
from scipy import stats

In [76]:
# Imports for better visualization
from matplotlib import rcParams
#colorbrewer2 Dark2 qualitative color table
dark2_colors = [(0.10588235294117647, 0.6196078431372549, 0.4666666666666667),
                (0.8509803921568627, 0.37254901960784315, 0.00784313725490196),
                (0.4588235294117647, 0.4392156862745098, 0.7019607843137254),
                (0.9058823529411765, 0.1607843137254902, 0.5411764705882353),
                (0.4, 0.6509803921568628, 0.11764705882352941),
                (0.9019607843137255, 0.6705882352941176, 0.00784313725490196),
                (0.6509803921568628, 0.4627450980392157, 0.11372549019607843)]

rcParams['figure.figsize'] = (10, 4)
rcParams['figure.dpi'] = 150
#rcParams['axes.color_cycle'] = dark2_colors
rcParams['lines.linewidth'] = 2
rcParams['font.size'] = 8
rcParams['patch.edgecolor'] = 'white'
rcParams['patch.facecolor'] = dark2_colors[0]
rcParams['font.family'] = 'StixGeneral'
rcParams['axes.grid'] = True
rcParams['axes.facecolor'] = '#eeeeee'


In [77]:
train = pd.read_csv('./data/train_5UKooLv.csv')

In [78]:
test = pd.read_csv('./data/test_J1hm2KQ.csv',usecols = train.columns)

In [79]:
test.head()

,Country,CustomerID,InvoiceDate,InvoiceNo,Quantity,StockCode,UnitPrice
0,PX,127269,01/12/10 8:28,127269,7,22633V,1.85
1,PX,227268,01/12/10 8:34,227268,38,84879M,1.69
2,PX,227268,01/12/10 8:34,227268,7,22748P,2.10
3,PX,227268,01/12/10 8:34,227268,9,22749K,3.75
4,PX,227268,01/12/10 8:34,227268,2,22622G,9.95


In [80]:
## In the test data, each user has certain number of reco, but that is just the 50% of his transactions,
## we have to predict the remaining 50% for each user}

In [81]:
submission_sample = pd.read_csv('./data/sample_submission_r7hs8Tz.csv')

In [82]:
train = train[test.columns]

### Reco PseudoTarget Analysis

In [83]:
print(test['StockCode'].nunique())
test['StockCode']

3522


0         22633V
1         84879M
2         22748P
3         22749K
4         22622G
           ...  
103092    23328X
103093    22556N
103094    22367K
103095    22629C
103096    23256O
Name: StockCode, Length: 103097, dtype: object

In [84]:
print(train['StockCode'].nunique())
train['StockCode']

3810


0         85123AY
1          71053R
2         84406BH
3         84029GV
4         84029EX
           ...   
330570     23498F
330571     22061J
330572     23275I
330573     21217W
330574     20685H
Name: StockCode, Length: 330575, dtype: object

In [85]:
test[test['StockCode'].isin(train['StockCode'].unique())]

,Country,CustomerID,InvoiceDate,InvoiceNo,Quantity,StockCode,UnitPrice
0,PX,127269,01/12/10 8:28,127269,7,22633V,1.85
1,PX,227268,01/12/10 8:34,227268,38,84879M,1.69
2,PX,227268,01/12/10 8:34,227268,7,22748P,2.10
3,PX,227268,01/12/10 8:34,227268,9,22749K,3.75
4,PX,227268,01/12/10 8:34,227268,2,22622G,9.95
5,PX,227268,01/12/10 8:34,227268,4,48187N,7.95
6,PX,327267,01/12/10 8:34,327267,3,22914W,4.95
7,PX,162738,01/12/10 9:02,162738,7,85123AY,2.55
8,PX,162738,01/12/10 9:02,162738,7,71053R,3.39
9,PX,162738,01/12/10 9:02,162738,9,84406BH,2.75


In [86]:
test[~test['StockCode'].isin(train['StockCode'].unique())]['StockCode'].nunique()

50

In [87]:
train[train['StockCode'].isin(test['StockCode'].unique())]

,Country,CustomerID,InvoiceDate,InvoiceNo,Quantity,StockCode,UnitPrice
0,PX,27270,01/12/10 8:26,27270,7,85123AY,2.55
1,PX,27270,01/12/10 8:26,27270,7,71053R,3.39
2,PX,27270,01/12/10 8:26,27270,9,84406BH,2.75
3,PX,27270,01/12/10 8:26,27270,7,84029GV,3.39
4,PX,27270,01/12/10 8:26,27270,7,84029EX,3.39
5,PX,27270,01/12/10 8:26,27270,2,22752G,7.65
6,PX,27270,01/12/10 8:26,27270,7,21730R,4.25
7,PX,427266,01/12/10 8:35,427266,3,21756Q,5.95
8,RA,462735,01/12/10 8:45,462735,28,22728B,3.75
9,RA,462735,01/12/10 8:45,462735,28,22727B,3.75


In [88]:
train[~train['StockCode'].isin(test['StockCode'].unique())]['StockCode'].nunique()

338

In [89]:
## 338 extra stockcodes in train, while 50 new in test

In [90]:
## extra stockcodes can be related by unit_price

In [91]:
train[train['Quantity']<=0]

,Country,CustomerID,InvoiceDate,InvoiceNo,Quantity,StockCode,UnitPrice
710,PX,69129,01/12/10 12:38,69129,-6,22960K,4.25
1070,PX,190908,01/12/10 14:30,190908,-1,22632O,2.10
1071,PX,190908,01/12/10 14:30,190908,-2,22355U,0.85
2048,GJ,309987,02/12/10 10:37,309987,-2,22752G,8.50
2098,PX,9990,02/12/10 10:46,9990,-2,22839T,14.95
2348,PX,290007,02/12/10 11:56,290007,-1,21463L,5.95
2521,PX,99099,02/12/10 12:50,99099,-4,22780U,4.25
2522,PX,99099,02/12/10 12:50,99099,-1,85048L,7.95
2524,PX,200898,02/12/10 13:05,200898,-2,22798I,2.95
2850,PX,220878,02/12/10 14:23,220878,-9360,84347V,0.03


In [92]:
test[test['Quantity']<=0]

,Country,CustomerID,InvoiceDate,InvoiceNo,Quantity,StockCode,UnitPrice
42,PX,342756,01/12/10 10:24,342756,-12,22556N,1.65
43,PX,342756,01/12/10 10:24,342756,-24,21983K,0.29
44,PX,342756,01/12/10 10:24,342756,-24,21980L,0.29
45,PX,342756,01/12/10 10:24,342756,-12,21484N,3.45
191,GJ,309087,01/12/10 14:33,309087,-4,22244X,1.95
192,GJ,309087,01/12/10 14:33,309087,-1,20914Q,2.95
193,GJ,309087,01/12/10 14:33,309087,-7,22892Z,1.25
194,GJ,309087,01/12/10 14:33,309087,-1,22654J,5.95
195,GJ,309087,01/12/10 14:33,309087,-2,22767S,9.95
196,GJ,309087,01/12/10 14:33,309087,-1,22333G,1.65


In [93]:
## lets assume a hypothesis
## Negative quantity means the customer didn't buy that product, might have been given it for free
## or returned it instaed

## Basic EDA

### Items Data

In [94]:
pd.Series(train['StockCode'].unique())

0       85123AY
1        71053R
2       84406BH
3       84029GV
4       84029EX
         ...   
3805    90182BN
3806    47591bU
3807     23843K
3808     23635S
3809     23644D
Length: 3810, dtype: object

In [95]:
codes = train['StockCode'].apply(lambda x:x[-2:])
codes = codes.apply(lambda x:x[1] if x[0].isdigit() else x)
print(codes.nunique())
codes

334


0         AY
1          R
2         BH
3         GV
4         EX
          ..
330570     F
330571     J
330572     I
330573     W
330574     H
Name: StockCode, Length: 330575, dtype: object

In [96]:
numeric_codes = train['StockCode'].apply(lambda x:x[:-1])
numeric_codes = numeric_codes.apply(lambda x:x[:-1] if x[-1].isdigit()==False else x)

In [97]:
primary_codes = numeric_codes.apply(lambda x:x[0:2])
secondary_codes = numeric_codes.apply(lambda x:x[2:4])
single_code = numeric_codes.apply(lambda x:x[4])

## Data Modelling

#### Two approaches could be followed
1. model the data on Quantity bough for each StockCode, considering the total quantity instead of individual transactions
2. Prepare a dummy variable, denoting whether the user bought that product or not, consider it as a binary target
3. at the end we can also try out the normalized_quantity approach but it mostly seems to fail.

#### Test data can be merged with train_data for training, and then predicting exact same number of products for each test user, as the test data is 50% transaction of each test_user, and we need to predict the remaining 50%

### Purchase count approach

### Train Test Merge

In [98]:
train.loc[train[train['Quantity']<0].index, 'Quantity'] *=-1
test.loc[test[test['Quantity']<0].index, 'Quantity'] *=-1

In [99]:
merged = pd.concat([train,test])
data_1 = merged.groupby(['CustomerID','Country','StockCode'])['StockCode'].agg({'total_quantity':'count'}).reset_index()

#### Item Data on Train-Test

In [100]:
price_data = merged.groupby(['StockCode'])['UnitPrice'].agg({'mode_price':scipy.stats.mode}).reset_index()
price_data['mode_price'] = price_data['mode_price'].apply(lambda x:x[0][0])

items_df = pd.DataFrame(merged['StockCode'].unique(),columns=['StockCode'])
codes = items_df['StockCode'].apply(lambda x:x[-2:])
items_df['alphabet_codes'] = codes.apply(lambda x:x[1] if x[0].isdigit() else x)

numeric_codes = items_df['StockCode'].apply(lambda x:x[:-1])
numeric_codes = numeric_codes.apply(lambda x:x[:-1] if x[-1].isdigit()==False else x)

items_df['primary_numeric'] = numeric_codes.apply(lambda x:x[0:2])
items_df['secondary_numeric'] = numeric_codes.apply(lambda x:x[2:4])
items_df['single_digit_numeric'] = numeric_codes.apply(lambda x:x[4])

items_df = items_df.merge(price_data,on=['StockCode'],how='inner')

#### Preprocess Item Data

In [101]:
data_1['total_quantity'].clip(upper=5,inplace=True)

In [102]:
data_1['total_quantity'].describe()

count    366405.000000
mean          1.182585
std           0.503004
min           1.000000
25%           1.000000
50%           1.000000
75%           1.000000
max           5.000000
Name: total_quantity, dtype: float64

#### Dummy data

In [103]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy

data_dummy = create_data_dummy(data_1)

#### normalized_quantity data

In [104]:
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='total_quantity', index='CustomerID', columns='StockCode')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['CustomerID'], value_name='scaled_purchase_freq').dropna()

In [105]:
data_norm = normalize_data(data_1)

### Turicreate model

In [106]:
### train_test_split
from sklearn.model_selection import train_test_split
### turicreate
import time
import turicreate as tc

In [107]:
def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [108]:
train_data, test_data = split_data(data_1)
train_data_dummy, test_data_dummy = split_data(data_dummy)

## Model Building - Validation

In [109]:
# constant variables to define field names include:
user_id = 'CustomerID'
item_id = 'StockCode'
users_to_recommend = list(train_data[user_id][0:3])
n_rec = 10 # number of items to recommend
n_display = 30 # to display the first few rows in an output dataset

In [110]:
def model(train_data, name, user_id, item_id, target, user_data, item_data, users_to_recommend, n_rec, n_display):
    if name == 'reco_fact':
        model = tc.ranking_factorization_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=None, solver='ials')

    elif name =='reco_fact_with_data':
        model = tc.ranking_factorization_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=None,user_data=user_data, item_data=item_data, solver='ials')
        
    
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [111]:
name = 'reco_fact_with_data'
target = 'total_quantity'
item_data = tc.SFrame(items_df.drop(['alphabet_codes','secondary_numeric','single_digit_numeric'],axis=1))
user_data = None
reco_fact_data = model(train_data, name, user_id, item_id, target, user_data, item_data, users_to_recommend, n_rec, n_display)

Warning: This solver currently does not support side features. Proceeding with training without side features.

Preparing data set.

Data has 293124 observations with 1597 users and 3830 items.

Data prepared in: 0.294525s

Training ranking_factorization_recommender for recommendations.

...

| 24      | 2.52s        | 0.630899                  |

| FINAL   | 2.52s        | 0.630899                  |

+---------+--------------+---------------------------+

Optimization Complete: Iteration limit reached.

+------------+-----------+--------------------+------+
| CustomerID | StockCode |       score        | rank |
+------------+-----------+--------------------+------+
|   362826   |   22727B  | 0.8316797614097595 |  1   |
|   362826   |   20725N  | 0.7542881369590759 |  2   |
|   362826   |   22720A  | 0.7171946167945862 |  3   |
|   362826   |   22086O  | 0.6843122839927673 |  4   |
|   362826   |   22728B  | 0.6635282635688782 |  5   |
|   362826   |   22470E  | 0.6497192978858948 |  6   |
|   362826   |   21175V  | 0.6497176289558411 |  7   |
|   362826   |   23209X  | 0.6338817477226257 |  8   |
|   362826   |  85099BJ  | 0.6151326894760132 |  9   |
|   362826   |   22867O  | 0.6067772507667542 |  10  |
|   176418   |  84596BQ  | 1.0722315311431885 |  1   |
|   176418   |   47566Y  | 1.0417845249176025 |  2   |
|   176418   |   23205F  | 1.0376877784729004 |  3   |
|   176418   |   22381J  | 0.9438562393188477 |  4   |
|   176418   |   20727V  | 0.934748649597168  |  5   |
|   176418

### Final Model

In [112]:
# constant variables to define field names include:
user_id = 'CustomerID'
item_id = 'StockCode'
users_to_recommend = list(train_data[user_id][0:3])
n_rec = 10 # number of items to recommend
n_display = 30 # to display the first few rows in an output dataset

In [113]:
target = 'total_quantity'
item_data = tc.SFrame(items_df.drop(['alphabet_codes','secondary_numeric','single_digit_numeric'],axis=1))
user_data = None

In [114]:
model = tc.ranking_factorization_recommender.create(tc.SFrame(data_dummy), 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target='total_quantity',user_data=user_data, item_data=item_data, 
                                                    solver='ials')
        


recom = model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

Warning: This solver currently does not support side features. Proceeding with training without side features.

Preparing data set.

Data has 366405 observations with 1600 users and 3860 items.

Data prepared in: 0.269992s

Training ranking_factorization_recommender for recommendations.

...

| 24      | 2.99s        | 0.848301                  |

| FINAL   | 2.99s        | 0.848301                  |

+---------+--------------+---------------------------+

Optimization Complete: Iteration limit reached.

+------------+-----------+--------------------+------+
| CustomerID | StockCode |       score        | rank |
+------------+-----------+--------------------+------+
|   362826   |   22960K  | 0.8227763772010803 |  1   |
|   362826   |   22470E  | 0.8025978803634644 |  2   |
|   362826   |   22728B  | 0.7989366054534912 |  3   |
|   362826   |   47566Y  | 0.7960784435272217 |  4   |
|   362826   |   22730Z  | 0.7202638983726501 |  5   |
|   362826   |   84879M  | 0.7172607183456421 |  6   |
|   362826   |   22720A  | 0.7144532799720764 |  7   |
|   362826   |   21175V  | 0.700600266456604  |  8   |
|   362826   |   22077P  | 0.6973388195037842 |  9   |
|   362826   |   22625P  | 0.6918052434921265 |  10  |
|   176418   |   21094A  | 1.2686989307403564 |  1   |
|   176418   |  84596BQ  | 1.1358253955841064 |  2   |
|   176418   |   20717I  | 1.018040418624878  |  3   |
|   176418   |   21080R  | 0.995925784111023  |  4   |
|   176418   |   21086B  | 0.9858379364013672 |  5   |
|   176418

In [117]:
model.save('final_model_submission')

## Model Testing on test data

For one user

In [44]:
customer = 127269
no_of_products = merged[merged['CustomerID']==customer]['StockCode'].nunique()
model.recommend(users=[127269], k=no_of_products)

CustomerID,StockCode,score,rank
127269,22909Z,0.9485517144203186,1
127269,21175V,0.9293605089187622,2
127269,85123AY,0.9239928722381592,3
127269,22423U,0.9160864353179932,4
127269,22144E,0.9137962460517883,5
127269,21166I,0.912585973739624,6
127269,22624O,0.8741899132728577,7
127269,84970SL,0.8570689558982849,8
127269,21181K,0.8566892743110657,9
127269,21914B,0.8536292314529419,10


#### For all users
So basically we will predict the remaining 50% products for each test customer, we are doing it by considering k = no of products bought before. Here I have one doubt, should we consider the 50% as prodcuts %, or the transaction %. I mean to say that if a customer did 500 transactions but bought only 400 products, so should we consider k=400 or k=500, because the contest page says that the test data is 50% transactions for a diffrent set of customers

In [45]:
test_user = pd.DataFrame(test['CustomerID'].unique(), columns=['CustomerID'])

In [46]:
test_user['no_reco_to_made'] = test_user['CustomerID'].apply(lambda x:merged[merged['CustomerID']==x]['StockCode'].nunique())

In [47]:
test_user['Items'] = test_user['CustomerID'].apply(lambda x: list(model.recommend(users=[x], k=int(test_user[test_user['CustomerID']==x]['no_reco_to_made']))['StockCode']))

#### Writing the submission file

In [48]:
test_user.drop('no_reco_to_made',axis=1).to_csv('./new_submissions_data_exp/reco_fact_target_purchase_count_with_item_user_new_experiment.csv',index=False,header=True)

In [49]:
# ohe = one hot encode

## Data Experiment logs

1. using only alphabetical, primary numeric, and 1e-5 regularization, leaderboard accuracy 0.2450, wont work

##### bc we have been using only 80% of data for experiment, and didnt use purchase count at all, thats why such low accuracies

2. Okay that worked fine, 0.2598 with all item data, and purchase count

3. Increased the iterations to 50, increased num_smapled_negative_examples to 10, didnt improve the performance, 0.2591

4. Used only primary_numeric codes, but didnt improve, came back to 0.2598

5. Removed all item data, used adagrad solver, and damn that took accuracy to 0.19 lololo!!!!!!!!! buoycott adagrad

6. One hot encoded primary_numeric, no improvement

# Ensemble Learning

### Lets try out ensemble here

For each user we predict required no. of products, then make it a single list, then find out top products from that list

Models to be used,
1. Model with item data and purchase count
2. Model without item data and purchase count
lets do it for two then we will see if this works or not

In [52]:
# constant variables to define field names include:
user_id = 'CustomerID'
item_id = 'StockCode'
users_to_recommend = list(train_data[user_id][0:3])
n_rec = 10 # number of items to recommend
n_display = 30 # to display the first few rows in an output dataset

target = 'total_quantity'
item_data = tc.SFrame(items_df.drop(['alphabet_codes','secondary_numeric','single_digit_numeric'],axis=1))
user_data = None

In [53]:
model_1 = tc.ranking_factorization_recommender.create(tc.SFrame(data_dummy), 
                                                    user_id=user_id, 
                                                    item_id=item_id, target='total_quantity',solver='ials')

model_2 = tc.ranking_factorization_recommender.create(tc.SFrame(data_dummy), 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target='total_quantity',user_data=user_data, item_data=item_data, 
                                                    solver='ials')

model_3 = tc.ranking_factorization_recommender.create(tc.SFrame(data_dummy), 
                                                    user_id=user_id, 
                                                    item_id=item_id, target='total_quantity',solver='ials',
                                                    ranking_regularization=0.1)

model_4 = tc.ranking_factorization_recommender.create(tc.SFrame(data_dummy), 
                                                    user_id=user_id, 
                                                    item_id=item_id, target='total_quantity',solver='ials',
                                                    max_iterations=75, regularization=1e-5)

model_5 = tc.ranking_factorization_recommender.create(tc.SFrame(data_dummy), 
                                                    user_id=user_id, 
                                                    item_id=item_id, target='total_quantity',item_data=item_data, 
                                                    solver='ials', num_sampled_negative_examples=20)


model_6 = tc.ranking_factorization_recommender.create(tc.SFrame(data_dummy), 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target='total_quantity',user_data=user_data, 
                                                    item_data=tc.SFrame(items_df.drop(['single_digit_numeric'],axis=1)), 
                                                    solver='ials')

model_7 = tc.ranking_factorization_recommender.create(tc.SFrame(data_dummy), 
                                                    user_id=user_id, 
                                                    item_id=item_id, target='total_quantity',solver='ials',
                                                    item_data=tc.SFrame(items_df.drop(['single_digit_numeric'],axis=1)),
                                                    max_iterations=75, regularization=1e-5)



Preparing data set.

Data has 366405 observations with 1600 users and 3860 items.

Data prepared in: 0.264924s

Training ranking_factorization_recommender for recommendations.

+------------------------------+--------------------------------------------------+----------+

...

| 73      | 11.03s       | 0.853636                  |

| 74      | 11.17s       | 0.85363                   |

| FINAL   | 11.17s       | 0.85363                   |

+---------+--------------+---------------------------+

Optimization Complete: Iteration limit reached.

In [54]:
test_user_new = pd.DataFrame(test['CustomerID'].unique(), columns=['CustomerID'])
test_user_new['no_reco_to_made'] = test_user_new['CustomerID'].apply(lambda x:merged[merged['CustomerID']==x]['StockCode'].nunique())

# test_user_new['Items'] = test_user_new['CustomerID'].apply(lambda x: list(model.recommend(users=[x], k=int(test_user_new[test_user_new['CustomerID']==x]['no_reco_to_made']))['StockCode']))

In [55]:
reco_series = pd.Series()
count = 0
for i in test_user_new['CustomerID']:
    no_of_products = merged[merged['CustomerID']==i]['StockCode'].nunique()
    
#     if no_of_products<10:
#         no_of_products=10
    
    m1 = pd.DataFrame(model_1.recommend(users=[i], k=(3860 - no_of_products)))
    m2 = pd.DataFrame(model_2.recommend(users=[i], k=(3860 - no_of_products)))
    m3 = pd.DataFrame(model_3.recommend(users=[i], k=(3860 - no_of_products)))
    m4 = pd.DataFrame(model_4.recommend(users=[i], k=(3860 - no_of_products)))
    m5 = pd.DataFrame(model_5.recommend(users=[i], k=(3860 - no_of_products)))
    m6 = pd.DataFrame(model_6.recommend(users=[i], k=(3860 - no_of_products)))
    m7 = pd.DataFrame(model_7.recommend(users=[i], k=(3860 - no_of_products)))    
    
    m1.columns = ['CustomerID','StockCode','rank_1','score_1']
    m2.columns = ['CustomerID','StockCode','rank_2','score_2']
    m3.columns = ['CustomerID','StockCode','rank_3','score_3']
    m4.columns = ['CustomerID','StockCode','rank_4','score_4']
    m5.columns = ['CustomerID','StockCode','rank_5','score_5']
    m6.columns = ['CustomerID','StockCode','rank_6','score_6']
    m7.columns = ['CustomerID','StockCode','rank_7','score_7']

    s1 = m1.merge(m2,on=['StockCode','CustomerID'])
    s2 = s1.merge(m3,on=['StockCode','CustomerID'])
    s3 = s2.merge(m4,on=['StockCode','CustomerID'])
    s4 = s3.merge(m5,on=['StockCode','CustomerID'])
    s5 = s4.merge(m6,on=['StockCode','CustomerID'])
    score_merged = s5.merge(m7,on=['StockCode','CustomerID']) 

    score_merged['weighted_score'] = (score_merged['score_1']+score_merged['score_2']+score_merged['score_3']+
                                 score_merged['score_4']+score_merged['score_5']+score_merged['score_6']+
                                  score_merged['score_7'])/7


    score_merged.sort_values('weighted_score',ascending=False,inplace=True)
    score_merged.reset_index(drop=True,inplace=True)
    ensemble_pred_list = score_merged['StockCode'].iloc[0:no_of_products]  
    
#     index = test_user_new.loc[test_user_new['CustomerID']==i].index[0]
#     test_user_new.loc[index,'Items'] = np.array(ensemble_pred_list)

    reco_series.loc[count]=list(ensemble_pred_list)
    
    count+=1

In [56]:
test_user_new['Items'] = reco_series

In [57]:
test_user_new.drop('no_reco_to_made',axis=1).to_csv('./new_submissions_data_exp/reco_fact_target_purchase_count_ensemble_with_item_user_new_experiment_more_user_data".csv',index=False,header=True)

## MORE FE THOUGHTS

1. Adding customer behaviour features, like customer byuing capabilaty, based on the min, max, var, count features of the products he bought previously.
2. Also the seasonality features like months and years he bought, his frequency of buying and all
2. Similarily adding product features like, customer 

In [58]:
train

,Country,CustomerID,InvoiceDate,InvoiceNo,Quantity,StockCode,UnitPrice
0,PX,27270,01/12/10 8:26,27270,7,85123AY,2.55
1,PX,27270,01/12/10 8:26,27270,7,71053R,3.39
2,PX,27270,01/12/10 8:26,27270,9,84406BH,2.75
3,PX,27270,01/12/10 8:26,27270,7,84029GV,3.39
4,PX,27270,01/12/10 8:26,27270,7,84029EX,3.39
5,PX,27270,01/12/10 8:26,27270,2,22752G,7.65
6,PX,27270,01/12/10 8:26,27270,7,21730R,4.25
7,PX,427266,01/12/10 8:35,427266,3,21756Q,5.95
8,RA,462735,01/12/10 8:45,462735,28,22728B,3.75
9,RA,462735,01/12/10 8:45,462735,28,22727B,3.75


In [59]:
user_quant = merged.groupby(['CustomerID'])['Quantity'].agg({'min_quant':'min','max_quant':'max','mean_quant':'mean','var_quant':'var','mode_quant':stats.mode}).reset_index()
user_quant['mode_quant'] = user_quant['mode_quant'].apply(lambda x:x[0][0])

user_country_1 = merged.groupby(['CustomerID'])['Country'].agg({'unique_countries':'unique','count_unique_countries':'nunique','mode_country':stats.mode}).reset_index()
user_country_1['mode_country'] = user_country_1['mode_country'].apply(lambda x:x[0][0])

user_price = merged.groupby(['CustomerID'])['UnitPrice'].agg({'min_price':'min','max_price':'max','mean_price':'mean','var_price':'var','mode_price':stats.mode}).reset_index()
user_price['mode_price'] = user_price['mode_price'].apply(lambda x:x[0][0])


items_quant = merged.groupby(['StockCode'])['Quantity'].agg({'min_quant':'min','max_quant':'max','mean_quant':'mean','var_quant':'var'}).reset_index()
items_price = merged.groupby(['StockCode'])['UnitPrice'].agg({'min_price':'min','max_price':'max','mean_price':'mean','var_price':'var'}).reset_index()

In [60]:
items_df = items_df.merge(items_price, on=['StockCode'],how='inner')
items_df = items_df.merge(items_quant, on=['StockCode'], how='inner')

In [61]:
user_df = pd.DataFrame(merged['CustomerID'].unique(),columns=['CustomerID'])

user_df = user_df.merge(user_quant, on=['CustomerID'], how='inner')
user_df = user_df.merge(user_country_1, on=['CustomerID'], how='inner')
user_df = user_df.merge(user_price, on=['CustomerID'], how='inner')

## Final Model experiment/

In [62]:
# constant variables to define field names include:
user_id = 'CustomerID'
item_id = 'StockCode'
users_to_recommend = list(train_data[user_id][0:3])
n_rec = 10 # number of items to recommend
n_display = 30 # to display the first few rows in an output dataset

In [63]:
target = 'total_quantity'
item_data = tc.SFrame(items_df)
user_data = tc.SFrame(user_df.drop(['unique_countries'],axis=1))

In [64]:
model = tc.ranking_factorization_recommender.create(tc.SFrame(data_dummy), 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target='total_quantity',user_data=user_data, item_data=item_data, 
                                                    solver='ials')
 

recom = model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

Warning: This solver currently does not support side features. Proceeding with training without side features.

Preparing data set.

Data has 366405 observations with 1600 users and 3860 items.

Data prepared in: 0.262377s

Training ranking_factorization_recommender for recommendations.

...

| 24      | 2.77s        | 0.848251                  |

| FINAL   | 2.77s        | 0.848251                  |

+---------+--------------+---------------------------+

Optimization Complete: Iteration limit reached.

+------------+-----------+--------------------+------+
| CustomerID | StockCode |       score        | rank |
+------------+-----------+--------------------+------+
|   36360    |   47566Y  | 0.6563539505004883 |  1   |
|   36360    |   22178V  | 0.6403002142906189 |  2   |
|   36360    |  85123AY  | 0.6083240509033203 |  3   |
|   36360    |   22697A  | 0.6005438566207886 |  4   |
|   36360    |   21314Q  | 0.5955486297607422 |  5   |
|   36360    |   84946F  | 0.5787147283554077 |  6   |
|   36360    |   22469S  | 0.5647217631340027 |  7   |
|   36360    |   22457C  | 0.5615686774253845 |  8   |
|   36360    |   21669G  | 0.5549391508102417 |  9   |
|   36360    |   22197J  | 0.5517067909240723 |  10  |
|   239058   |   22966E  | 0.9547631740570068 |  1   |
|   239058   |   22197J  | 0.9052513837814331 |  2   |
|   239058   |   22625P  | 0.8967427015304565 |  3   |
|   239058   |   21931F  | 0.8942598700523376 |  4   |
|   239058   |   21080R  | 0.8866696953773499 |  5   |
|   239058

In [65]:
model_1 = tc.ranking_factorization_recommender.create(tc.SFrame(data_dummy), 
                                                    user_id=user_id, 
                                                    item_id=item_id, target='total_quantity',solver='ials')

model_2 = tc.ranking_factorization_recommender.create(tc.SFrame(data_dummy), 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target='total_quantity',user_data=user_data, item_data=item_data, 
                                                    solver='ials')

model_3 = tc.ranking_factorization_recommender.create(tc.SFrame(data_dummy), 
                                                    user_id=user_id, 
                                                    item_id=item_id, target='total_quantity',solver='ials',
                                                    ranking_regularization=0.1)

model_4 = tc.ranking_factorization_recommender.create(tc.SFrame(data_dummy), 
                                                    user_id=user_id, 
                                                    item_id=item_id, target='total_quantity',solver='ials',
                                                    max_iterations=75, regularization=1e-5)

model_5 = tc.ranking_factorization_recommender.create(tc.SFrame(data_dummy), 
                                                    user_id=user_id, 
                                                    item_id=item_id, target='total_quantity',
                                                    user_data=user_data, item_data=item_data, 
                                                    solver='ials', num_sampled_negative_examples=20)


model_6 = tc.ranking_factorization_recommender.create(tc.SFrame(data_dummy), 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target='total_quantity',user_data=user_data, 
                                                    item_data=item_data, 
                                                    solver='ials')

model_7 = tc.ranking_factorization_recommender.create(tc.SFrame(data_dummy), 
                                                    user_id=user_id, 
                                                    item_id=item_id, target='total_quantity',solver='ials',
                                                    user_data=user_data,
                                                      item_data=item_data,
                                                    max_iterations=75, regularization=1e-5)


model_8 = tc.ranking_factorization_recommender.create(tc.SFrame(data_dummy), 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=None, 
                                                    solver='sgd')
 

Preparing data set.

Data has 366405 observations with 1600 users and 3860 items.

Data prepared in: 0.25075s

Training ranking_factorization_recommender for recommendations.

+------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training Predictive Error.

Final objective value: 1.3248

Final training Predictive Error: 0.657928

In [66]:
test_user_new = pd.DataFrame(test['CustomerID'].unique(), columns=['CustomerID'])
test_user_new['no_reco_to_made'] = test_user_new['CustomerID'].apply(lambda x:merged[merged['CustomerID']==x]['StockCode'].nunique())

# test_user_new['Items'] = test_user_new['CustomerID'].apply(lambda x: list(model.recommend(users=[x], k=int(test_user_new[test_user_new['CustomerID']==x]['no_reco_to_made']))['StockCode']))

In [67]:
reco_series = pd.Series()
count = 0
for i in test_user_new['CustomerID']:
    no_of_products = merged[merged['CustomerID']==i]['StockCode'].nunique()
    
#     if no_of_products<10:
#         no_of_products=10
    
    m1 = pd.DataFrame(model_1.recommend(users=[i], k=(3860 - no_of_products)))
    m2 = pd.DataFrame(model_2.recommend(users=[i], k=(3860 - no_of_products)))
    m3 = pd.DataFrame(model_3.recommend(users=[i], k=(3860 - no_of_products)))
    m4 = pd.DataFrame(model_4.recommend(users=[i], k=(3860 - no_of_products)))
    m5 = pd.DataFrame(model_5.recommend(users=[i], k=(3860 - no_of_products)))
    m6 = pd.DataFrame(model_6.recommend(users=[i], k=(3860 - no_of_products)))
    m7 = pd.DataFrame(model_7.recommend(users=[i], k=(3860 - no_of_products)))    
    
    m1.columns = ['CustomerID','StockCode','rank_1','score_1']
    m2.columns = ['CustomerID','StockCode','rank_2','score_2']
    m3.columns = ['CustomerID','StockCode','rank_3','score_3']
    m4.columns = ['CustomerID','StockCode','rank_4','score_4']
    m5.columns = ['CustomerID','StockCode','rank_5','score_5']
    m6.columns = ['CustomerID','StockCode','rank_6','score_6']
    m7.columns = ['CustomerID','StockCode','rank_7','score_7']

    s1 = m1.merge(m2,on=['StockCode','CustomerID'])
    s2 = s1.merge(m3,on=['StockCode','CustomerID'])
    s3 = s2.merge(m4,on=['StockCode','CustomerID'])
    s4 = s3.merge(m5,on=['StockCode','CustomerID'])
    s5 = s4.merge(m6,on=['StockCode','CustomerID'])
    score_merged = s5.merge(m7,on=['StockCode','CustomerID']) 

    score_merged['weighted_score'] = (score_merged['score_1']+score_merged['score_2']+score_merged['score_3']+
                                 score_merged['score_4']+score_merged['score_5']+score_merged['score_6']+
                                  score_merged['score_7'])/7


    score_merged.sort_values('weighted_score',ascending=False,inplace=True)
    score_merged.reset_index(drop=True,inplace=True)
    ensemble_pred_list = score_merged['StockCode'].iloc[0:no_of_products]  
    
#     index = test_user_new.loc[test_user_new['CustomerID']==i].index[0]
#     test_user_new.loc[index,'Items'] = np.array(ensemble_pred_list)

    reco_series.loc[count]=list(ensemble_pred_list)
    
    count+=1

In [68]:
test_user_new['Items'] = reco_series
test_user_new.drop('no_reco_to_made',axis=1).to_csv('./new_submissions_data_exp/reco_fact_target_purchase_count_ensemble_with_item_user_new_experiment_more_user_data_1".csv',index=False,header=True)

###### best results as of now, ensemble after adding item data, user data. Used the ensemble method in previous snippet, not the one right above this, and got a score of 0.262654